In [1]:
import os
import sys
from pathlib import Path

# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv()
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings(model="embedding-passage")

DB_URI = os.environ["POSTGRES_URI"]     
# from langchain_postgres import PGEngine, PGVectorStore
# pg_engine = PGEngine.from_connection_string(
#     url=DB_URI
# )

from langchain_postgres import PGVector
from langchain_core.documents import Document
vector_store = PGVector(
    embeddings=embeddings,
    collection_name="SPRI_ALL",
    connection=DB_URI
)

In [2]:
import pickle
papers = []
for path in sorted(Path(project_root).joinpath("outputs").glob("*_split_documents.pkl")):
    
    with open(path, "rb") as f:
        split_documents = pickle.load(f)
        for doc in split_documents:
            doc.metadata["title"] = path.stem.replace("_output_split_documents", "")
    papers.append(split_documents)


In [3]:
len(papers)

3

In [4]:
papers[1][0].metadata

{'page': 1,
 'image_id': [],
 'image_path': [],
 'text_summary': [],
 'image_summary': [],
 'id': 0,
 'title': 'SPRI_2023'}

In [9]:
for paper in papers:
    vector_store.add_documents(paper)

In [10]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10, "filter": 
{"$and": [
            {"title": {"$eq": "SPRI_2023"}},
                        
        ]}
})

In [11]:
retriever.invoke("AI 트렌드")

[Document(id='f334780d-d1c9-42f5-9b44-0ced5304567e', metadata={'id': 95, 'page': 54, 'title': 'SPRI_2023', 'image_id': [], 'image_path': [], 'text_summary': '요약:\n\n- 제목과 맥락\n  - SPRi 이슈리포트 IS-159는 AI Index 2023의 주요 내용과 시사점을 정리.\n\n- 정부 전략의 이행 가속화\n  - 국가초고성능컴퓨팅 혁신전략(21.5.28), 국산 AI반도체를 활용한 K-클라우드 추진방안(22.12.12), 초거대 AI 경쟁력 강화방안(23.4.14) 등 정부 추진 전략의 단계별 이행 가속 필요.\n\n- AI 윤리와 규제\n  - AI 오용 사례 증가와 생성 AI 확산으로 AI 윤리에 대한 관심 증가.\n  - 공정성/편향 등 신뢰성 검증을 위한 구체적 방안 필요.\n  - 규제와 책임 등 정부 역할 중요.\n  - 활용 분야별 주요 지표 파악과 명확한 기준 제시 요구.\n  - 모델 규모에 따른 편향/독성은 학습 데이터와 완화 방법에 따라 달라질 수 있으며, 공정한 모델일수록 덜 편향될 가능성.\n  - 편향성과 독성 완화를 위한 가이드라인 필요: 명령 조정, 데이터 필터링 등.\n\n- 경제·투자 흐름\n  - 세계 경기 침체 영향으로 AI 투자·도입은 감소하나, 한국은 투자 확대를 통해 6위권 유지 및 세계 시장 비중 제고 필요.\n  - 글로벌 AI 투자 규모, 건수, 피투자기업 수 모두 전년 대비 감소.\n  - 10억 달러 이상 대형 투자를 제외한 건수 감소로 신규 피투자 기업 수 감소 및 건당 투자액 증가하는 집중화 현상 심화.\n  - 미국의 영향력 여전히 강함: 미국은 전 세계 AI 스타트업 투자의 약 52%를 차지하고 AI 채용 공고 비중도 높아 시장 지배력 강화.\n  - 중국은 AI 민간투자 규모·스타트업 수 등에서 2위 유지; 영국, 캐나다, 이스라엘, 독일 등도 상위 국가로 지속.\n

In [17]:
from langchain_community.retrievers import BM25Retriever

In [9]:
docs = []
for paper in papers:
    docs.extend(paper)

In [10]:
len(docs)

247

In [12]:
from sqlalchemy import create_engine, text

engine = create_engine(DB_URI)
def sql_query() -> list[tuple[str, dict]]:
    with engine.connect() as conn:
        # Custom SQL query
        query = conn.execute(text(f"""
            SELECT 
                e.document,
                e.cmetadata            
            FROM langchain_pg_embedding e
            JOIN langchain_pg_collection c ON e.collection_id = c.uuid
            WHERE c.name = :collection_name
                AND e.cmetadata->>'title' = :title
        """), {
            "collection_name": "SPRI_ALL",
            "title": "SPRI_2023",
        })
        
        return query.fetchall()

In [21]:
result = sql_query()
arr = []
for r in result:
    arr.append(Document(page_content=r[0], metadata=r[1]))

In [14]:
arr =sorted(arr, key=lambda x: x.metadata["id"])

In [15]:
len(arr)

108

In [22]:
bm25_retriever = BM25Retriever.from_documents(arr)
bm25_retriever.k = 10

In [23]:
bm25_retriever.invoke("AI 트렌드")

[Document(metadata={'id': 38, 'page': 23, 'title': 'SPRI_2023', 'image_id': [265, 269], 'image_path': ['/images/SPRI_2023_cropped_chart_265.png', '/images/SPRI_2023_cropped_table_269.png'], 'text_summary': '다음은 제공된 텍스트의 요약입니다:\n\n- 주제: SPRi 이슈리포트 IS-159로, AI Index 2023의 주요 내용과 시사점을 다룸.\n- 근거: Stanford HAI의 AI Index Report 2023 자료를 인용.\n- 핵심 트렌드: 2012년부터 2021년까지 AI 사고·논쟁 수가 지속적으로 증가해 왔으며, 차트상 2012년 5건에서 2021년 260건으로 증가.\n- 주의사항: 2022년 보고된 사건은 심사과정으로 아직 완전히 반영되지 않아 미고려.\n- 2022년 대표 사례(요약): \n  - 볼로디미르 젤렌스키 항복 영상: 딥페이크로 확인된 가짜 영상의 확산.\n  - Verus 교도소 수감자 통화 모니터링: AI 음성인식으로 감시, 개인정보보호 및 차별 우려 제기.\n  - 인텔의 학생 감정 모니터링 시스템 개발: Zoom에서 학생 감정 식별 시도, 개인정보보호와 차별 문제 제기.\n  - 런던 경찰청 갱 폭력 매트릭스: 1,000명대 데이터 세트를 기반으로 위험도 측정, 정확성 저하 및 소수민족 차별 가능성 지적.\n  - Midjourney: 텍스트로 이미지 생성 서비스. 저작권 학습 데이터, 고용 대체, 개인정보보호 이슈 제기.\n- 요지: 2022년 사례들은 개인정보보호, 차별, 저작권, 고용 등에 관한 윤리적·법적 문제를 부각시키며 AI 활용의 리스크를 강조.', 'image_summary': ['그림은 2012년부터 2021년까지 연도별 AI 사고 및 논쟁의 건수를 나타내는 막대그래프이다. x축은 연도, y축은 사건 수를 표시한다. 초기(2012~2014년)는

In [20]:
retriever.invoke("AI 트렌드")

[Document(id='f334780d-d1c9-42f5-9b44-0ced5304567e', metadata={'id': 95, 'page': 54, 'title': 'SPRI_2023', 'image_id': [], 'image_path': [], 'text_summary': '요약:\n\n- 제목과 맥락\n  - SPRi 이슈리포트 IS-159는 AI Index 2023의 주요 내용과 시사점을 정리.\n\n- 정부 전략의 이행 가속화\n  - 국가초고성능컴퓨팅 혁신전략(21.5.28), 국산 AI반도체를 활용한 K-클라우드 추진방안(22.12.12), 초거대 AI 경쟁력 강화방안(23.4.14) 등 정부 추진 전략의 단계별 이행 가속 필요.\n\n- AI 윤리와 규제\n  - AI 오용 사례 증가와 생성 AI 확산으로 AI 윤리에 대한 관심 증가.\n  - 공정성/편향 등 신뢰성 검증을 위한 구체적 방안 필요.\n  - 규제와 책임 등 정부 역할 중요.\n  - 활용 분야별 주요 지표 파악과 명확한 기준 제시 요구.\n  - 모델 규모에 따른 편향/독성은 학습 데이터와 완화 방법에 따라 달라질 수 있으며, 공정한 모델일수록 덜 편향될 가능성.\n  - 편향성과 독성 완화를 위한 가이드라인 필요: 명령 조정, 데이터 필터링 등.\n\n- 경제·투자 흐름\n  - 세계 경기 침체 영향으로 AI 투자·도입은 감소하나, 한국은 투자 확대를 통해 6위권 유지 및 세계 시장 비중 제고 필요.\n  - 글로벌 AI 투자 규모, 건수, 피투자기업 수 모두 전년 대비 감소.\n  - 10억 달러 이상 대형 투자를 제외한 건수 감소로 신규 피투자 기업 수 감소 및 건당 투자액 증가하는 집중화 현상 심화.\n  - 미국의 영향력 여전히 강함: 미국은 전 세계 AI 스타트업 투자의 약 52%를 차지하고 AI 채용 공고 비중도 높아 시장 지배력 강화.\n  - 중국은 AI 민간투자 규모·스타트업 수 등에서 2위 유지; 영국, 캐나다, 이스라엘, 독일 등도 상위 국가로 지속.\n